In [1]:
import sys
import torch
import time
import torch.backends.cudnn as cudnn
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [2]:
# Parameters

data_path = './data/UCSD_Anomaly_Dataset.v1p2/UCSDped2/'
output = './output/UCSDped2/'

test_path = data_path + 'Test/'
train_path = data_path + 'Train/'
output_test_path = output + 'Test/'
output_train_path = output + 'Train/'

DAE_in_size = 64
min_box_size_ratio = 0.002
valid_perc = 0.1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# load the model
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')

# define confidence of the model
model.conf = 0.3  # confidence threshold (0-1)
model.iou = 0.5   # NMS IoU threshold (0-1)

# define classes here
model.classes = [0, 1, 2, 3, 5, 6, 7, 8, 14, 15, 16]

Using cache found in C:\Users\basarbatu/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-11-3 torch 1.10.0+cu113 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients, 109.1 GFLOPs
Adding AutoShape... 


In [4]:
train_folders = [train_path+x for x in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, x))]
test_folders = [test_path+x for x in os.listdir(test_path) if os.path.isdir(os.path.join(test_path, x)) and x[-2:]!='gt']
images = [x for x in os.listdir(train_folders[0]) if x[0]!='.']
input_img_path = train_folders[0] + '/' + images[0]
img = cv2.imread(input_img_path)
min_box_size = img.shape[0]*img.shape[1]*min_box_size_ratio
print(min_box_size)

172.8


In [5]:
d = {'xmin': [], 'ymin': [], 'xmax': [], 'ymax': [], 'area': [], 'confidence':[], 'class':[], 'name':[], 'ori_im_path':[], 'scaled_im_path':[], 'label':[], 'frame_n':[], 'train_valid_test':[]}
data_meta = pd.DataFrame(data=d)
frame_index = 0
yolo_obj_index = 0
# reading train images
for fold in tqdm(train_folders):
    images = [x for x in os.listdir(fold) if x[0]!='.']
    for i in range(0, len(images)):
        # read image
        input_img_path = fold + '/' + images[i]
        img = cv2.imread(input_img_path)
        # make prediction
        results = model(img)
        out = results.pandas().xyxy[0]
        # calculate area
        out['area'] = (out['xmax']-out['xmin'])*(out['ymax']-out['ymin'])
        # update common info
        out['train_valid_test'] = 'train'
        out['frame_n'] = frame_index
        out['ori_im_path'] = input_img_path
        # append meta data
        data_meta = data_meta.append(out, ignore_index=True)
        # go through the all detections
        for j in range(0, len(out)):
            # get the cropped image
            xmin = int(out.loc[j, 'xmin'])
            xmax = int(out.loc[j, 'xmax'])
            ymin = int(out.loc[j, 'ymin'])
            ymax = int(out.loc[j, 'ymax'])
            # extract image
            yolo_object_img = img[ymin:ymax, xmin:xmax, :]
            # rescale the cropped image
            resized_yolo_object_img = cv2.resize(yolo_object_img, (DAE_in_size, DAE_in_size), interpolation = cv2.INTER_AREA)
            # save the scaled images
            saved_path = output_train_path + '{:06d}.png'.format(yolo_obj_index)
            cv2.imwrite(saved_path, resized_yolo_object_img)
            data_meta.loc[yolo_obj_index, 'scaled_im_path'] = saved_path
            # update meta data for label
            data_meta.loc[yolo_obj_index, 'label'] = 0
            yolo_obj_index = yolo_obj_index+1
        frame_index = frame_index+1
print('Object detection for training images completed...')

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [01:49<00:00,  6.87s/it]

Object detection for training images completed...


In [6]:
# Assign last percentage of the training set as validation

n_valid = round(len(data_meta)*valid_perc)
valid_indices = data_meta.sample(n_valid).index
data_meta.loc[valid_indices, 'train_valid_test'] = 'valid'

In [8]:
# reading test images
for fold in tqdm(test_folders):
    images = [x for x in os.listdir(fold) if x[0]!='.']
    for i in range(0, len(images)):
        # read image
        input_img_path = fold + '/' + images[i]
        img = cv2.imread(input_img_path)
        # read annot
        input_annot_path = fold + '_gt/' + images[i].split('.')[0]+'.bmp'
        label = cv2.imread(input_annot_path, cv2.COLOR_BGR2GRAY)
        # for some cases in USCDped2, there label is starting with frame
        if label is None:
            input_annot_path = fold + '_gt/frame' + images[i].split('.')[0]+'.bmp'
            label = cv2.imread(input_annot_path, cv2.COLOR_BGR2GRAY)
        # for some cases there will not be ground truth
        if label is None:
            label = img[:,:,0]*0
        # make prediction
        results = model(img)
        out = results.pandas().xyxy[0]
        # calculate area
        out['area'] = (out['xmax']-out['xmin'])*(out['ymax']-out['ymin'])
        # update common info
        out['train_valid_test'] = 'test'
        out['frame_n'] = frame_index
        out['ori_im_path'] = input_img_path
        # append meta data
        data_meta = data_meta.append(out, ignore_index=True)
        # go through the all detections
        for j in range(0, len(out)):
            # get the cropped image
            xmin = int(out.loc[j, 'xmin'])
            xmax = int(out.loc[j, 'xmax'])
            ymin = int(out.loc[j, 'ymin'])
            ymax = int(out.loc[j, 'ymax'])
            # extract image
            yolo_object_img = img[ymin:ymax, xmin:xmax, :]
            yolo_object_label = label[ymin:ymax, xmin:xmax]
            # rescale the cropped image
            resized_yolo_object_img = cv2.resize(yolo_object_img, (DAE_in_size, DAE_in_size), interpolation = cv2.INTER_AREA)
            # save the scaled images
            saved_path = output_test_path + '{:06d}.png'.format(yolo_obj_index)
            cv2.imwrite(saved_path, resized_yolo_object_img)
            data_meta.loc[yolo_obj_index, 'scaled_im_path'] = saved_path
            # update meta data for label
            is_abnormal = np.sum(yolo_object_label/255)/(yolo_object_label.shape[0]*yolo_object_label.shape[1])
            data_meta.loc[yolo_obj_index, 'label'] = is_abnormal
            yolo_obj_index = yolo_obj_index+1
        frame_index = frame_index+1
print('Object detection for test images completed...')

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [01:56<00:00,  9.72s/it]

Object detection for test images completed...


In [9]:
# Shuffle train valid and test sets within their groups
train_data = data_meta[data_meta['train_valid_test']=='train'].sample(frac=1)
valid_data = data_meta[data_meta['train_valid_test']=='valid'].sample(frac=1)
test_data = data_meta[data_meta['train_valid_test']=='test'].sample(frac=1)
data_meta = pd.concat([train_data, valid_data, test_data], ignore_index=True)

In [10]:
# Exclude the small area yolo objects
n1 = data_meta.shape[0]
data_meta = data_meta[data_meta['area']>min_box_size]
n2 = data_meta.shape[0]
print('Total row number dropped from {} to {}'.format(n1,n2))
# Save the meta file
data_meta.to_csv(output + 'meta_data.csv', index=False)

Total row number dropped from 60268 to 58456
